# MLflow Ragas Evaluation Example

This notebook demonstrates how to use `mlflow.evaluate()` with a custom Ragas evaluator plugin that runs evaluations via Kubeflow Pipelines.

## Prerequisites

- Have a Kubeflow Pipelines endpoint available.
- Setup env vars in .env file.
- Have access to the S3 bucket used for storing evaluation results.

## Imports

In [1]:
from rich import print as rprint
import mlflow
import pandas as pd

## MLflow setup
Run MLflow locally with
```bash
mlflow ui --backend-store-uri sqlite:///mlflow.db --port 5000
```


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

Ensure that our entry-point in pyproject.toml gets picked up by the MFflow plugin discovery. You should see ragas_kubeflow in the list of available evaluators:

In [3]:
rprint(mlflow.models.list_evaluators())
assert "ragas_kubeflow" in mlflow.models.list_evaluators()

/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['default', 'classifier', 'regressor', 'shap', 'ragas_kubeflow']

## Make sure things run: test with a static dataset using MLflow's default evaluator


In [4]:
# Evaluation with static predictions
eval_data_with_predictions = pd.DataFrame(
    {
        "inputs": [
            "What is the capital of France?",
            "How does photosynthesis work?",
            "Explain machine learning in simple terms",
        ],
        "context": [
            "France is a country in Europe with Paris as its capital city.",
            "Photosynthesis is the process by which plants use sunlight to make food.",
            "Machine learning is a subset of AI that learns patterns from data.",
        ],
        "ground_truth": [
            "Paris",
            "Photosynthesis",
            "Machine learning",
        ],
        "predictions": [
            "Paris",
            "Photosynthesis",
            "Machine learning",
        ],
    }
)

with mlflow.start_run() as run:
    results = mlflow.evaluate(
        data=eval_data_with_predictions,
        targets="ground_truth",
        predictions="predictions",
        extra_metrics=[mlflow.metrics.genai.answer_similarity()],
        evaluators=["default"],
    )

    print("✅ Static dataset evaluation completed!")
    print(f"   Run ID: {run.info.run_id}")

    # Show metrics
    print("\n📊 Metrics:")
    for metric_name, value in results.metrics.items():
        print(f"   {metric_name}: {value:.4f}")

2025/09/05 15:37:47 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
100%|██████████| 1/1 [00:00<00:00, 32.60it/s]
/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:4268: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:180: RuntimeWarning: invalid value encountered in 

✅ Static dataset evaluation completed!
   Run ID: 964657ffc1374fb985648be01b9bb97d

📊 Metrics:
   answer_similarity/v1/mean: nan
   answer_similarity/v1/variance: nan
🏃 View run rumbling-dove-216 at: http://localhost:5000/#/experiments/0/runs/964657ffc1374fb985648be01b9bb97d
🧪 View experiment at: http://localhost:5000/#/experiments/0


## Evaluation over a static dataset with our custom pipeline runner on KFP

In [5]:
# this will invoke the Ragas evaluator, so the dataset
# should conform to the expected format for Ragas
evaluation_data = pd.DataFrame.from_records(
    [
        {
            "user_input": "What is the capital of France?",
            "response": "The capital of France is Paris.",
            "retrieved_contexts": [
                "Paris is the capital and most populous city of France."
            ],
            "reference": "Paris",
        },
        {
            "user_input": "What is photosynthesis?",
            "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
            "retrieved_contexts": [
                "Photosynthesis is a process used by plants to convert light energy into chemical energy."
            ],
            "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
        },
    ]
)
rprint(evaluation_data)

user_input  \
0  What is the capital of France?   
1         What is photosynthesis?   

                                            response  \
0                    The capital of France is Paris.   
1  Photosynthesis is the process by which plants ...   

                                  retrieved_contexts  \
0  [Paris is the capital and most populous city o...   
1  [Photosynthesis is a process used by plants to...   

                                           reference  
0                                              Paris  
1  Photosynthesis is the process by which plants ...

In [ ]:
with mlflow.start_run() as run:
    mlflow.evaluate(
        data=evaluation_data,
        model_type="ragas",  # This just needs to pass can_evaluate()
        evaluators=["ragas_kubeflow"],  # This explicitly selects our evaluator
        # evaluator_config={...}, # Can override config here
    )

/Users/dmanilof/src/trustyai-llm-demo/ragas-quickstart-demo/.venv/lib/python3.12/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


<mlflow.data.evaluation_dataset.EvaluationDataset object at 0x30d36acc0>